# Image distortion notebook

In [1]:
import torch
import numpy as np
import os
import shutil
from datetime import datetime
import math
from math import exp
from random import seed
import random as rand
import matplotlib.pyplot as plt
import pickle 
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import cdist
import time

import sys
sys.path.append('../common/')
from utils import *
from optimisation import *
from loo_cv import *
from image_commons import *
from scipy.ndimage import map_coordinates

In [2]:
seed=0
torch.manual_seed(seed)
np.random.seed(seed)

In [3]:
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from torch.jit import script, trace       # hybrid frontend decorator and tracing jit
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [4]:
class Net(nn.Module):
    def __init__(self,N=10):
            super(Net, self).__init__()
            
            nInputs = int(N*N/2-N/2)
            
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(in_features=nInputs, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=32),
                nn.ReLU(),
                nn.Linear(in_features=32, out_features=16),
                nn.ReLU(),
                nn.Linear(in_features=16, out_features=1)
            )
            # map to positive
            
            
    def forward(self, x):
            logits = self.linear_relu_stack(x)
            return logits

# Generate features

In [5]:
def upper_half(A):
    n = A.shape[0]
    return 1/A[np.triu_indices(n, k = 1)]

In [6]:
# network to be tested:
n=10
path =   '../network/results_sorted_1d_2d/best_model.pt'
model=Net()
model.load_state_dict(torch.load(path))
model.eval()

Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=45, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=16, bias=True)
    (9): ReLU()
    (10): Linear(in_features=16, out_features=1, bias=True)
  )
)

# Distortion

In [9]:
# Apply barrel distortion to the image
def apply_barrel_distortion(image, k1=0.3):
    # Normalize image coordinates to the range [-1, 1]
    h, w = image.shape
    x, y = np.meshgrid(np.linspace(-1, 1, w), np.linspace(-1, 1, h))
    r = np.sqrt(x**2 + y**2)
    
    # Apply barrel distortion
    x_distorted = x * (1 + k1 * r**2)
    y_distorted = y * (1 + k1 * r**2)
    
    # Map the distorted coordinates back to image coordinates
    x_distorted = ((x_distorted + 1) / 2) * (w - 1)
    y_distorted = ((y_distorted + 1) / 2) * (h - 1)
    
    # Use map_coordinates to interpolate the image at the distorted coordinates
    distorted_image = map_coordinates(image, [y_distorted.flatten(), x_distorted.flatten()], order=1).reshape(h, w)
    return distorted_image, x_distorted, y_distorted

# Use RBF interpolation to correct the distortion
def correct_distortion_rbf(distorted_image, x_distorted, y_distorted):
    h, w = distorted_image.shape
    # Generate the original grid coordinates
    x_original, y_original = np.meshgrid(np.arange(w), np.arange(h))
    
    # Flatten the arrays for processing
    x_original_flat = x_original.flatten()
    y_original_flat = y_original.flatten()
    x_distorted_flat = x_distorted.flatten()
    y_distorted_flat = y_distorted.flatten()
    
    # Use a subset of points for the RBF due to computational constraints
    # Select every Nth point to reduce the number of points
    N = 5  # Adjust N for a balance between performance and accuracy
    indices = np.arange(0, len(x_original_flat), N)
    
    # Prepare the data for RBF
    points_distorted = np.vstack((x_distorted_flat[indices], y_distorted_flat[indices]))
    points_original = np.vstack((x_original_flat[indices], y_original_flat[indices]))
    points_x = np.vstack((x_original_flat, y_original_flat)).T
    points_original1=points_original.T
    points_distorted1=points_distorted.T
    # Create RBF interpolators for x and y mappings
    p_ref=np.array([0,0])    
    no_evaluation_points=x_original_flat.shape[0]
    distance =cdist(points_x,points_distorted1)
    closest = np.argpartition(distance, 10, axis=1)
    scaled_training_set_flat = []
    eps_final_hardy=np.zeros((no_evaluation_points,1))
    eps_final_franke=np.zeros((no_evaluation_points,1))
    eps_final_mfranke=np.zeros((no_evaluation_points,1))
    eps_final_rippa=np.zeros((no_evaluation_points,2))
    eps_final_mle=np.zeros((no_evaluation_points,2))
    
    x_coords = []
    f=[]
    x_corrected_flat=np.zeros((6,no_evaluation_points))
    y_corrected_flat=np.zeros((6,no_evaluation_points))
    for i in range(no_evaluation_points):
        x_local=np.zeros((10,2))
        fpart=np.zeros((10+1,2))
        for j in range(10):
            x_local[j]=points_distorted1[closest[i][j]]
            fpart[j]=points_original1[closest[i][j]]
        x_local = sorted( x_local, key = lambda x: np.linalg.norm(x - p_ref ) )
        x_local = np.reshape(x_local, (10,2)) 
        x_coords.append(x_local)
        f.append(fpart)
        x_axis=x_local.reshape(10,2)
        # Hardy approach
        nearest_distances = find_nearest_neighbors(x_axis)
        d=np.sum(nearest_distances)/n        
        eps_final_hardy[i]=1/(0.815*d)

        # Franke approach
        x_axis_list=x_axis.tolist()        
        circle = find_min_circle(x_axis_list)
        eps_final_franke[i]=0.8*(10**(1/2))/(2 * circle[2])

        # Modified Franke approach
        eps_final_mfranke[i]=0.8*(10**(1/4))/(2 * circle[2])

        # Rippa approach       
        eps_v = [0.001, 0.002, 0.005, 0.0075, 0.01, 0.02, 0.05, 0.075, \
                 0.1, 0.2, 0.5, 0.75,1,2, 5, 7.5, 10, 20.0, 50, 75, 100, 200, 500, 1000]     
        # 
        ## x-axis
        rhs_rippa = fpart[:10][:,0]
        best_eps = 0
        old_error = np.inf        
        for eps in eps_v:
            error_r = rippa_cv(eps, x_axis, rhs_rippa)
            if error_r < old_error:
               best_eps = eps
               old_error = error_r
        eps_final_rippa[i,0]=best_eps

        # y-axis
        rhs_rippa = fpart[:10][:,1]
        best_eps = 0
        old_error = np.inf        
        for eps in eps_v:
            error_r = rippa_cv(eps, x_axis, rhs_rippa)
            if error_r < old_error:
               best_eps = eps
               old_error = error_r
        eps_final_rippa[i,1]=best_eps

        # MLE approach
        rhs_rippa = fpart[:10][:,0]
        best_eps = 1.0
        old_error = np.inf        
        for eps in eps_v:
            error_r = mle_cv(eps, x_axis, rhs_rippa)
            if error_r < old_error:
               best_eps = eps
               old_error = error_r
        eps_final_mle[i,0]=best_eps
        # y-axis
        rhs_rippa = fpart[:10][:,1]
        best_eps = 1.0
        old_error = np.inf        
        for eps in eps_v:
            error_r = mle_cv(eps, x_axis, rhs_rippa)
            if error_r < old_error:
               best_eps = eps
               old_error = error_r
        eps_final_mle[i,1]=best_eps
        
        # NN approach
        xxx=generate_distance_from_coordinates(x_axis)               
        training_set_distances_flatten = upper_half(xxx)
        scaled_training_set_flat.append(training_set_distances_flatten)
        if i%1000==0:
            print(i)
        

    scaled_training_set_tensor = torch.tensor(scaled_training_set_flat,dtype=torch.float)
    scaled_training_set_tensor.reshape((no_evaluation_points,int(10*10/2-10/2)))
        
    eps_final= model(scaled_training_set_tensor)
    eps_final_optimisation = eps_final.detach().numpy()

    for j in range(no_evaluation_points):
        for count,eps in enumerate([eps_final_mle[j],eps_final_rippa[j],eps_final_hardy[j][0]\
                                    ,eps_final_franke[j][0],\
                                    eps_final_mfranke[j][0],eps_final_optimisation[j][0]]):

            if count==0 or count==1: #choose ripa or mle
                Lx =  get_int_matrix(x_coords[j],eps[0])
                Ly =  get_int_matrix(x_coords[j],eps[1])
                fx=f[j][:,0]
                fy=f[j][:,1]

                wx=np.linalg.solve(Lx, fx) 
                wy=np.linalg.solve(Ly, fy) 
    
                sx=np.zeros((1,10+1))
                sy=np.zeros((1,10+1))
                for i in range(10):      
                    sx[0,i] = phi(eps[0],points_x[j] , x_coords[j][i])
                    sy[0,i] = phi(eps[1],points_x[j] , x_coords[j][i])
                sx[0,-1]=1.0 
                sy[0,-1]=1.0 
   
                x_corrected_flat[count][j]=np.matmul(sx,wx)        
                y_corrected_flat[count][j]=np.matmul(sy,wy)

            else:
                L =  get_int_matrix(x_coords[j],eps)
                
                fx=f[j][:,0]
                fy=f[j][:,1]

                wx=np.linalg.solve(L, fx) 
                wy=np.linalg.solve(L, fy) 
    
                s=np.zeros((1,10+1))
                for i in range(10):      
                    s[0,i] = phi(eps,points_x[j] , x_coords[j][i])
                s[0,-1]=1.0        
   
                x_corrected_flat[count][j]=np.matmul(s,wx)        
                y_corrected_flat[count][j]=np.matmul(s,wy)

    corrected_image=[]
    for count in range(6):
        # Ensure the coordinates are within image bounds
        x_corrected_flat[count] = np.clip(x_corrected_flat[count], 0, w - 1)
        y_corrected_flat[count] = np.clip(y_corrected_flat[count], 0, h - 1)
    
        # Use map_coordinates to get the corrected image
        corrected_image.append(map_coordinates(distorted_image, [y_corrected_flat[count], x_corrected_flat[count]], order=1).reshape(h, w))
    return corrected_image

# Checkerboard

In [ ]:
original_image = create_checkerboard(size=(200, 200))  # Create a synthetic checkerboard image
distorted_image, x_distorted, y_distorted = apply_barrel_distortion(original_image)  # Apply barrel distortion
corrected_image = correct_distortion_rbf(distorted_image, x_distorted, y_distorted)  # Correct the distortion

fig1 = plt.figure(figsize=(10, 6))
plt.imshow(original_image, cmap='gray')
plt.axis('off')
plt.savefig(f'original_checkerboard',bbox_inches='tight', dpi=150)
plt.close(fig1)

fig2 = plt.figure(figsize=(10, 6))
plt.imshow(distorted_image, cmap='gray')
plt.axis('off')
plt.savefig(f'distorted_checkerboard',bbox_inches='tight', dpi=150)
plt.close(fig2)

strategies=['MLE','Rippa','Hardy','Franke','Modified Franke', 'NN']

mse=[0 for i in range(6)]
psnr=[0 for i in range(6)]

for count in range(6):
    print('Shape parameter strategy:',strategies[count])
    display_images(corrected_image[count],strategies[count]+'_distortion_checkerboard')  # Display the images
    mse[count] = np.mean((original_image - corrected_image[count]) ** 2)
    psnr[count]=20*np.log10(np.max(original_image)/np.sqrt(mse[count]))

print('mse',mse)
print('psnr',psnr)

0
1000


/users/PAS2708/hanveiga1/new_rbf/RBF-shape-parameter-NN/2-dimensional/../common/loo_cv.py:50: RuntimeWarning: invalid value encountered in log
  term1 = np.log(np.dot(rhs2,w)+1e-16)


2000


# Flowers

In [13]:
path = 'flower.png'
original_image = load_image(path)
distorted_image, x_distorted, y_distorted = apply_barrel_distortion(original_image)  # Apply barrel distortion
corrected_image = correct_distortion_rbf(distorted_image, x_distorted, y_distorted)  # Correct the distortion

fig1 = plt.figure(figsize=(10, 6))
plt.imshow(original_image, cmap='gray')
plt.axis('off')
plt.savefig(f'original_flower',bbox_inches='tight', dpi=150)
plt.close(fig1)

fig2 = plt.figure(figsize=(10, 6))
plt.imshow(distorted_image, cmap='gray')
plt.axis('off')
plt.savefig(f'distorted_flower',bbox_inches='tight', dpi=150)
plt.close(fig2)

strategies=['MLE','Rippa','Hardy','Franke','Modified Franke', 'NN']

mse=[0 for i in range(6)]
psnr=[0 for i in range(6)]

for count in range(6):
    print('Shape parameter strategy:',strategies[count])
    display_images(corrected_image[count],strategies[count]+"_distortion_flower")  # Display the images
    mse[count] = np.mean((original_image - corrected_image[count]) ** 2)
    psnr[count]=20*np.log10(np.max(original_image)/np.sqrt(mse[count]))

print('mse',mse)
print('psnr',psnr)

/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/2-dimensional/image/../../common/loo_cv.py:51: RuntimeWarning: invalid value encountered in log
  term2 = np.log(np.linalg.det(M)+1e-16)


0
1000


/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/2-dimensional/image/../../common/loo_cv.py:50: RuntimeWarning: invalid value encountered in log
  term1 = np.log(np.dot(rhs2,w)+1e-16)


2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000


/tmp/slurmtmp.672334/ipykernel_3483328/3065281354.py:150: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  scaled_training_set_tensor = torch.tensor(scaled_training_set_flat,dtype=torch.float)


[[2.2963371]
 [2.2955346]
 [2.2942011]
 ...
 [2.2999425]
 [2.2976205]
 [2.2967668]] [[0.1 0.1]
 [0.1 0.1]
 [0.1 0.1]
 ...
 [0.1 0.1]
 [0.1 0.1]
 [0.1 0.1]]
Shape parameter strategy: MLE
Shape parameter strategy: Rippa
Shape parameter strategy: Hardy
Shape parameter strategy: Franke
Shape parameter strategy: Modified Franke
Shape parameter strategy: NN
mse [0.04829213969189625, 0.022922648401014437, 0.0064938067279571, 0.010202850320608513, 0.022342162797972482, 0.004253296261518399]
psnr [12.919465525637822, 16.155582077553262, 21.633236419997463, 19.67101485337724, 16.266977887748446, 23.47097366421899]
